In [ ]:
!nvidia-smi

Sat Apr 22 15:06:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb --quiet
!pip install torchsummaryX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torch import nn
from torchsummaryX import summary
import os
import gc
from tqdm import tqdm
import numpy as np
import pandas as pd
import glob
import wandb
import matplotlib.pyplot as plt
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)
import json
import gc

Device:  cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from google.colab import drive # Link your drive if you are a colab user
# drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

In [ ]:
# npz_dict = {}
# for file in os.listdir("/content/val_data"):
#   a = file.split(".")[0]
#   arr = np.load(os.path.join("/content/val_data",file))

#   npz_dict[a] = arr.tolist()

In [ ]:
# npz_dict

In [ ]:
# with open("val_data.json", "w") as outfile:
#     json.dump(npz_dict, outfile)

In [ ]:
# a = open("/content/ViT_L_14@336px_train.json")
# resnet = json.load(a)
# print(len(sorted(list(resnet.keys()))))

# print(len(sorted(os.listdir("/content/train_data"))))

In [ ]:
# # !mkdir '/content/data'

# !unzip -qo 'train_data.zip' -d '/content'


# # !unzip -qo '/content/train_data.zip' -d '/content/data'

# Configs

In [ ]:
config = {
    'batch_size': 10, # Increase this if your GPU can handle it
    'lr': 0.00005,
    # 'lr': 0.0001,
    'epochs': 35, # 10 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
    # Include other parameters as needed.
}

#Dataset

In [ ]:
class Projectdataset(torch.utils.data.Dataset):

    def __init__(self, root, train_val = "train", caption_type = "ViT_L_14@336px"): # Feel free to add more arguments

        self.vid_feat, self.cap_feat = [], []

        self.vid_dir_train       = os.path.join(root,"train"+"_data")
        self.cap_dir_train = os.path.join(root,caption_type+"_"+"train"+".json")

        self.vid_dir_val       = os.path.join(root,"val"+"_data")
        self.cap_dir_val = os.path.join(root,caption_type+"_"+"val"+".json")

        cap_dir_train = open(self.cap_dir_train)
        cap_json_train = json.load(cap_dir_train)

        cap_dir_val = open(self.cap_dir_val)
        cap_json_val = json.load(cap_dir_val)

        cap_names_train   = sorted(list(cap_json_train.keys()))
        cap_names_val   = sorted(list(cap_json_val.keys()))

        vid_names_train    = sorted(os.listdir(self.vid_dir_train))
        vid_names_val    = sorted(os.listdir(self.vid_dir_val))

        assert len(vid_names_train) + len(vid_names_val) == len(cap_names_train) + len(cap_names_val) 

        # TODO: Iterate through mfccs and transcripts
        for i in cap_names_train:
            vid        = np.load(os.path.join(self.vid_dir_train,i+".npy")).squeeze()
            cap  = cap_json_train[i]["Text"]
            cap = np.array(cap).squeeze().mean(axis=0)
            self.vid_feat.append(vid)
            self.cap_feat.append(cap)
        
        for i in cap_names_val:
            vid        = np.load(os.path.join(self.vid_dir_val,i+".npy")).squeeze()
            cap  = cap_json_val[i]["Text"]
            cap = np.array(cap).squeeze().mean(axis=0)

            self.vid_feat.append(vid)
            self.cap_feat.append(cap)


        self.length = len(self.vid_feat)
        cap_dir_train.close()
        cap_dir_val.close()

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        vid = self.vid_feat[ind]
        cap = self.cap_feat[ind]

        vid      = torch.FloatTensor(vid) 
        cap      = torch.FloatTensor(cap)       

        return vid,cap 


In [ ]:
root = r"/content/drive/MyDrive/"
train_data = Projectdataset(root, train_val = "train", caption_type = "ViT_L_14@336px")
val_data = Projectdataset(root, train_val = "val", caption_type = "ViT_L_14@336px")

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = False,
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False,
)

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))

Train dataset samples = 1790, batches = 179


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
torch.cuda.device_count()

1

In [ ]:
for i,data in enumerate(train_loader):

    vid, cap = data
    print("video data shape is: ", vid.shape)
    print("caption data shape is: ", cap.shape)
    break

video data shape is:  torch.Size([10, 200, 3072])
caption data shape is:  torch.Size([10, 768])


## Models

In [ ]:
class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

class Linear1(torch.nn.Module):
    def __init__(self, input_size=3072, output_size=768):

        super(Linear1, self).__init__()
        self.name = "linear 1"
        
        self.linear1 = torch.nn.Linear(input_size, input_size)
        self.linear2 = torch.nn.Linear(input_size, input_size)
        self.linear3 = torch.nn.Linear(input_size, input_size)
        self.gelu    = torch.nn.GELU()
        self.dropout = torch.nn.Dropout(p=0.15)
        self.bn1     = torch.nn.BatchNorm1d(input_size)
        self.bn2     = torch.nn.BatchNorm1d(input_size)
        self.downsize = torch.nn.Linear(input_size, output_size) 
        self.sigmoid = torch.nn.Sigmoid()     
        self.permute = PermuteBlock()
    def forward(self, x):
        x = self.linear1(x)
        x = self.permute(x)
        x = self.permute(self.bn1(x))
        x = self.dropout(self.gelu(x))
        x = self.linear2(x)
        x = self.permute(x)
        x = self.permute(self.bn2(x))
        x = self.gelu(x)
        out = torch.mean(x, 1)
        out = self.downsize(self.dropout(out))
        return out


class model_conv1(torch.nn.Module):
  def __init__(self, input_size=3072, output_size=768):
    super(model_conv1,self).__init__()
    self.name = "model_conv1"
    self.conv = nn.Sequential(
        torch.nn.Conv1d(input_size,input_size,kernel_size=5, padding=2, groups = input_size),
        torch.nn.GELU(), ## Added this
        torch.nn.BatchNorm1d(num_features=3072), ## Added this
        torch.nn.Conv1d(input_size,input_size,kernel_size=5, padding=2, groups = input_size)
        )
    self.gelu = torch.nn.GELU()
    self.dropout = torch.nn.Dropout(p=0.15)
    self.downsize = torch.nn.Linear(input_size,output_size)
    self.permute = PermuteBlock()

  def forward(self, x):
    x = self.permute(x)
    x = self.conv(x)
    x = self.permute(x)
    out = torch.mean(x, 1)
    out = self.dropout(self.downsize(out))
    return out


class model_conv1_linear(torch.nn.Module):
  def __init__(self, input_size=3072, output_size=768):
    super(model_conv1_linear,self).__init__()
    self.name = "model_conv1"
    self.conv = nn.Sequential(
        torch.nn.Conv1d(input_size,input_size,kernel_size=5, padding=2, groups = input_size),
        torch.nn.BatchNorm1d(input_size),
        torch.nn.GELU(),
        torch.nn.Conv1d(input_size,input_size,kernel_size=5, padding=2, groups = input_size)
        )
    self.gelu = torch.nn.GELU()
    self.dropout = torch.nn.Dropout(p=0.15)
    self.bn = torch.nn.BatchNorm1d(input_size)
    self.linear = torch.nn.Linear(input_size, input_size)
    self.downsize = torch.nn.Linear(input_size,output_size)
    self.permute = PermuteBlock()
  def forward(self, x):
    x = self.permute(x)
    x = self.bn(self.conv(x))
    x = self.permute(x)
    x = self.dropout(x)
    x = self.linear(x)
    x = self.gelu(x)
    out = torch.mean(x, 1)
    out = self.dropout(self.downsize(out))
    return out


class model_conv2(torch.nn.Module):
  def __init__(self, input_size=3072, output_size=768):
    super(model_conv2,self).__init__()
    self.name = "model_conv2"
    self.conv = nn.Sequential(
        torch.nn.Conv1d(200,200,kernel_size=5, padding=2, groups=200),
        torch.nn.BatchNorm1d(200),
        torch.nn.GELU(),
        torch.nn.Conv1d(200,200,kernel_size=5, padding=2, groups=200)
        )
    self.gelu = torch.nn.GELU()
    self.dropout = torch.nn.Dropout(p=0.15)
    self.downsize = torch.nn.Linear(input_size,output_size)

  def forward(self, x):
    x = self.conv(x)
    out = torch.mean(x, 1)
    out = self.dropout(self.downsize(out))
    return out


In [ ]:
class Linear1_imp(torch.nn.Module):
    def __init__(self, input_size=3072, output_size=768):

        super(Linear1_imp, self).__init__()
        self.name = "linear 1"
        
        self.linear1 = torch.nn.Linear(input_size, input_size)
        self.linear2 = torch.nn.Linear(input_size, input_size)
        self.gelu    = torch.nn.GELU()
        self.dropout = torch.nn.Dropout(p=0.15)
        self.bn1     = torch.nn.BatchNorm1d(input_size)
        self.bn2     = torch.nn.BatchNorm1d(input_size)
        self.permute = PermuteBlock()
    def forward(self, x):
        x = self.linear1(x)
        x = self.permute(x)
        x = self.permute(self.bn1(x))
        x = self.dropout(self.gelu(x))
        x = self.linear2(x)
        x = self.permute(x)
        x = self.permute(self.bn2(x))
        return x


class model_conv1_imp_bn(torch.nn.Module):
  def __init__(self, input_size=3072, output_size=768):
    super(model_conv1_imp_bn,self).__init__()
    self.name = "model_conv1_imp"
    self.conv = torch.nn.Sequential(
        torch.nn.Conv1d(input_size,input_size,kernel_size=5, padding=2, groups = input_size),
        torch.nn.GELU(), ## Added this
        torch.nn.BatchNorm1d(num_features=3072), ## Added this
        torch.nn.Conv1d(input_size,input_size,kernel_size=5, padding=2, groups = input_size)
        )
    self.permute = PermuteBlock()
  def forward(self, x):
    x = self.permute(x)
    x = self.conv(x)
    x = self.permute(x)
    return x

class model_conv1_linear_imp(torch.nn.Module):
  def __init__(self, input_size=3072, output_size=768):
    super(model_conv1_linear_imp,self).__init__()
    self.name = "model_conv1_update2"
    self.conv = torch.nn.Sequential(
        torch.nn.Conv1d(input_size,input_size,kernel_size=5, padding=2, groups = input_size),
        torch.nn.BatchNorm1d(3072),
        torch.nn.GELU(),
        torch.nn.Conv1d(input_size,input_size,kernel_size=5, padding=2, groups = input_size)
        )
    self.linear = torch.nn.Linear(input_size, input_size)
    self.bn = torch.nn.BatchNorm1d(input_size)
    self.permute = PermuteBlock()

  def forward(self, x):
    x = self.permute(x)
    x = self.bn(self.conv(x))
    x = self.permute(x)
    out = self.linear(x)
    return out

class model_conv2_imp(torch.nn.Module):
  def __init__(self, input_size=3072, output_size=768):
    super(model_conv2_imp,self).__init__()
    self.name = "model_conv2_imp"
    self.conv = torch.nn.Sequential(
        torch.nn.Conv1d(200,200,kernel_size=5, padding=2, groups=200),
        torch.nn.BatchNorm1d(200),
        torch.nn.GELU(),
        torch.nn.Conv1d(200,200,kernel_size=5, padding=2, groups=200)
        )
  def forward(self, x):
    x = self.conv(x)
    return x


In [ ]:
DEVICE

'cuda'

In [ ]:
gc.collect()
torch.cuda.empty_cache()
model = Linear1().to(DEVICE)
summary(model, vid.to(DEVICE))


             Kernel Shape     Output Shape     Params  Mult-Adds
Layer                                                           
0_linear1    [3072, 3072]  [10, 200, 3072]  9.440256M  9.437184M
1_permute               -  [10, 3072, 200]          -          -
2_bn1              [3072]  [10, 3072, 200]     6.144k     3.072k
3_permute               -  [10, 200, 3072]          -          -
4_gelu                  -  [10, 200, 3072]          -          -
5_dropout               -  [10, 200, 3072]          -          -
6_linear2    [3072, 3072]  [10, 200, 3072]  9.440256M  9.437184M
7_permute               -  [10, 3072, 200]          -          -
8_bn2              [3072]  [10, 3072, 200]     6.144k     3.072k
9_permute               -  [10, 200, 3072]          -          -
10_gelu                 -  [10, 200, 3072]          -          -
11_dropout              -       [10, 3072]          -          -
12_downsize   [3072, 768]        [10, 768]  2.360064M  2.359296M
-------------------------

/usr/local/lib/python3.9/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_linear1,"[3072, 3072]","[10, 200, 3072]",9440256.0,9437184.0
1_permute,-,"[10, 3072, 200]",NaN,NaN
2_bn1,[3072],"[10, 3072, 200]",6144.0,3072.0
3_permute,-,"[10, 200, 3072]",NaN,NaN
4_gelu,-,"[10, 200, 3072]",NaN,NaN
5_dropout,-,"[10, 200, 3072]",NaN,NaN
6_linear2,"[3072, 3072]","[10, 200, 3072]",9440256.0,9437184.0
7_permute,-,"[10, 3072, 200]",NaN,NaN
8_bn2,[3072],"[10, 3072, 200]",6144.0,3072.0


#Only save the tuner part of the models

In [ ]:
model = Linear1_imp()
save_path ="model.pth"
torch.save({'model_state_dict':model.state_dict()},'/content/'+save_path)

In [ ]:
pretrained_check_point = torch.load("/content/model1_run.pth")
model_check_point = torch.load("/content/model.pth")

In [ ]:
pretrained_dict = pretrained_check_point["model_state_dict"]
model_dict = model_check_point["model_state_dict"]

In [ ]:
pretrained_dict

In [ ]:
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict)
# 3. load the new state dict
model.load_state_dict(model_dict)

<All keys matched successfully>

In [ ]:
model_dict

In [ ]:
###Save the actual model state dict to be used for PDVC
save_path = "ACTUAL_Linear1_imp.pth"
torch.save({'model_state_dict':model.state_dict()},
                  '/content/drive/MyDrive/pth files final/'+save_path)

In [ ]:
model.state_dict()

# Optimizers, scheduler and loss




In [ ]:
# criterion = torch.nn.MSELoss()

criterion = torch.nn.CosineEmbeddingLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr= config['lr'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 30, eta_min = 0.000005)
scaler = torch.cuda.amp.GradScaler()

# Train Funtion

In [ ]:
def train(model, dataloader, optimizer, criterion):
    
    model.train()

    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 

    num_correct = 0
    total_loss  = 0

    for i, (vid, caption) in enumerate(dataloader):
        
        optimizer.zero_grad()

        vid, caption =vid.to(DEVICE), caption.to(DEVICE)
        
        with torch.cuda.amp.autocast(): 
            output = model(vid)
            batch, _ = output.shape
            ## for cosine similarity
            target = torch.ones(batch).to(DEVICE)
            loss    = criterion(output, caption, target)
            ## for MSE
            # loss    = criterion(output, caption)
        total_loss      += float(loss.item())

        batch_bar.set_postfix(
            loss        = "{:.04f}".format(float(total_loss / (i + 1))),
            lr          = "{:.04f}".format(float(optimizer.param_groups[0]['lr']))
        )
        
        scaler.scale(loss).backward() 
        scaler.step(optimizer)
        scaler.update() 
      
        batch_bar.update() 
        del vid, caption, output, loss
    batch_bar.close() 

    total_loss  = float(total_loss / len(dataloader))

    return total_loss

In [ ]:
def validate(model, dataloader, criterion):
  
    model.eval()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)
    
    total_loss = 0.0

    for i, (vid, caption) in enumerate(dataloader):

        vid, caption =vid.to(DEVICE), caption.to(DEVICE)
        

        with torch.inference_mode():

            output = model(vid)
            batch, _ = output.shape
            ## for cosine similarity
            target = torch.ones(batch).to(DEVICE)
            loss = criterion(output, caption, target)
            ## for mse
            # loss = criterion(output, caption)

        total_loss += float(loss.item())

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))))

        batch_bar.update()
        
    batch_bar.close()
    total_loss = float(total_loss / len(dataloader))
    print("total_loss:", total_loss)
    return total_loss

In [ ]:
gc.collect()
torch.cuda.empty_cache()

# Wandb

In [ ]:
wandb.login(key="4db423a16f4cfa4122b415b4c149bab53147a4cb") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name = "project_Linear1_actual", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # id = "vfrm12ve", ##Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "IDL_Project", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: shangs (idlbeasts). Use `wandb login --relogin` to force relogin


# Training

In [ ]:
best_train = 10

for epoch in range(config['epochs']):

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_loss = train(model, train_loader, optimizer, criterion)
    
    print("\nEpoch {}/{}: \n Train Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1,
        config['epochs'],
        train_loss,
        curr_lr))
    
    # val_loss = validate(model, val_loader, criterion)


    # print("val loss:", val_loss)

    # wandb.log({"train_loss":train_loss,
    #   'validation_loss': val_loss, "learning_Rate": curr_lr})
    
    wandb.log({"train_loss":train_loss, "learning_Rate": curr_lr})

    if epoch >= 4:  
      scheduler.step()

    if train_loss <= best_train:
      print("Saving model")
      save_path = "model1_run.pth"
      torch.save({'model_state_dict':model.state_dict(),
                  # 'scheduler_state_dict':scheduler.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict()},
                  '/content/'+save_path)
      best_train = train_loss
      wandb.save(save_path)
run.finish()